### Лабораторная работа №2. Реализация глубокой нейронной сет

In [1]:
import os
import random
import tarfile
from collections import defaultdict

tar_path = 'notMNIST_large.tar.gz'
tar_general_dir = 'notMNIST_large'
total_images = 500000 + 19000
train_size = 500000 // 10
test_size = 19000 // 10

**Задание 1.** Реализуйте полносвязную нейронную сеть с помощью библиотеки Tensor Flow. В качестве алгоритма оптимизации можно использовать, например, стохастический градиент (Stochastic Gradient Descent, SGD). Определите количество скрытых слоев от 1 до 5, количество нейронов в каждом из слоев до нескольких сотен, а также их функции активации (кусочно-линейная, сигмоидная, гиперболический тангенс и т.д.).

Загрузим все изображения из архива

In [2]:
with tarfile.open(tar_path, 'r:gz') as tar:
    tar.extractall(path='.')

image_paths = defaultdict(list)

for folder in os.listdir(tar_general_dir):
    folder_path = os.path.join(tar_general_dir, folder)
    if os.path.isdir(folder_path):
        for img_file in os.listdir(folder_path):
            image_paths[folder].append(os.path.join(folder_path, img_file))

Разделим изображения две выборки: обучающую (500 тыс. изображений) и тестовую (19 тыс. изображений)

In [4]:
train_list, test_list = [], []
for class_images in image_paths.values():
    random.shuffle(class_images)
    train_list.extend(class_images[:train_size])
    test_list.extend(class_images[train_size:train_size + test_size])